# Chapter 10  
## Modules and Packages

Modules and packages are the core of any large project, and the Python installation itself.  
This chapter focuses on common programming techniques involving modules and packages, such as how to organize packages, splitting large modules into multiple files, and creating namespace packages.  
Recipes that allow you to customize the operation of the `import` statement itself are also given.

## 10.1. Making a Hierarchical Package of Modules

### Problem

You want to organize your code into a package consisting of a hierarchical collection of modules.

### Solution

Making a package structure is simple.  
Just organize your code as you wish on the filesystem and make sure that every directory defines an `__init__.py` file:

Once you have done this, you should be able to perform various `import` statements such as the following:

### Discussion

Defining a hierarchy of modules is as easy as making a directory structure on the filesystem.  
The purpose of the `__init__.py` files is to include optional initialization code that runs as different levels of a package are encountered.  
For example, if you have the statement `import graphics`, the file *graphics/`__init__`.py* will be imported and form the contents of the `graphics` namespace.  
For an import such as `import graphics.formats.jpg`, the files *graphics/`__init__`.py* and *graphics/formats/`__init__`.py* will both be imported prior to the final import of the `graphics/formats/jpg.py` file.  
More often that not, it’s fine to just leave the `__init__.py` files empty. 
However, there are certain situations where they might include code.  
For example, an `__init__.py` file can be used to automatically load submodules like this:

For such a file, a user merely has to use a single `import graphics.formats` instead of a separate import for `graphics.formats.jpg` and `graphics.formats.png`.  
Other common uses of `__init__.py` include consolidating definitions from multiple files into a single logical namespace, as is sometimes done when splitting modules.  
This is discussed in Recipe 10.4.  
Astute programmers will notice that Python 3.3 still seems to perform package imports even if no `__init__.py` files are present.  
If you don’t define `__init__.py`, you actually create what’s known as a "namespace package" which is described in Recipe 10.5.  
All things being equal, include the `__init__.py` files if you’re just starting out with the creation of a new package.

## 10.2. Controlling the Import of Everything

### Problem

You want precise control over the symbols that are exported from a module or package when a user uses the `from module import *` statement.

### Solution

Define a variable `__all__` in your module that explicitly lists the exported names.

In [4]:
# somemodule.py

def spam():
    pass

def grok():
    pass

blah = 42

# Only export 'spam' and 'grok':
__all__ = ['spam', 'grok']

### Discussion

Although the use of `from module import *` is strongly discouraged, it still sees frequent use in modules that define a large number of names.  
If you don’t do anything, this form of import will export all names that don’t start with an underscore.  
On the other hand, if `__all__` is defined, then only the names explicitly listed will be exported.  
If you define `__all__` as an empty list, then nothing will be exported.  
An `AttributeError` is raised on import if `__all__` contains undefined names.

## 10.3. Importing Package Submodules Using Relative Names

### Problem

You have code organized as a package and want to import a submodule from one of the other package submodules without hardcoding the package name into the import statement.

### Solution

To import modules of a package from other modules in the same package, use a package-relative import.  
For example, suppose you have a package `mypackage` organized as follows on the filesystem:

If the module `mypackage.A.spam` wants to import the module `grok` located in the same directory, it should include an import statement like this:

If the same module wants to import the module `B.bar` located in a different directory, it can use an import statement like this:

Both of the import statements shown operate relative to the location of the *spam.py* file and do not include the top-level package name.

### Discussion

Inside packages, imports involving modules in the same package can either use fully specified absolute names or a relative import using the syntax shown:

The downside of using an absolute name, such as `mypackage.A`, is that it hardcodes the top-level package name into your source code.  
This, in turn, makes your code more brittle and hard to work with if you ever want to reorganize it.  
For example, if you ever changed the name of the package, you would have to go through all of your files and fix the source code.  
Similarly, hardcoded names make it difficult for someone else to move the code around.  
For example, perhaps someone wants to install two different versions of a package, differentiating them only by name.  
If relative imports are used, it would all work fine, whereas everything would break with absolute names.  
The `.` and `..` syntax on the `import` statement might look funny, but think of it as specifying a directory name.  
`.` means look in the current directory and `..B` means look in the *../B* directory.  
This syntax only works with the `from` form of import.

Although it looks like you could navigate the filesystem using a relative import, they are not allowed to escape the directory in which a package is defined.  
That is, combinations of dotted name patterns that would cause an import to occur from a non-package directory cause an error.  
Finally, it should be noted that relative imports only work for modules that are located inside a proper package.  
In particular, they do not work inside simple modules located at the top level of scripts.  
They also won’t work if parts of a package are executed directly as a script:

On the other hand, if you execute the preceding script using the `-m` flag, the relative imports will work properly:

For more information on relative package imports, see [PEP 328](https://www.python.org/dev/peps/pep-0328/)

## 10.4. Splitting a Module into Multiple Files